<a href="https://colab.research.google.com/github/RJaeschke1982/Codex-Playground/blob/main/EBM_Compass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#@title EBM Compass 🧭
#@markdown ---
#@markdown ### **Krok 1: Wpisz temat i uruchom tę komórkę**
#@markdown Wpisz poniżej swój temat badawczy. Narzędzie automatycznie zainstaluje zależności i przeprowadzi cały proces.
TEMAT_BADAWCZY = "" #@param {type:"string"}
#@markdown ---

# ===================================================================
# EBM Compass - Wersja Finalna (Autonomous Research Agent)
# -------------------------------------------------------------------
# Narzędzie do automatycznej syntezy i rankingu dowodów medycznych.
# Operator: dr n. med. Rafał Jaeschke
# ===================================================================
import sys
import subprocess
import io, json, openai, os, textwrap, time, csv
from datetime import datetime

# --- Automatyczna Instalacja Zależności ---
def ensure_dependencies():
    """Weryfikuje i instaluje brakujące pakiety."""
    print("--- [SETUP] Weryfikacja zależności... ---")
    required = {'biopython', 'openai', 'pandas', 'tqdm'}
    try:
        installed_raw = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'], stderr=subprocess.DEVNULL)
        installed = {pkg.split('==')[0].lower() for pkg in installed_raw.decode().split('\n')}
        missing = required - installed
        if missing:
            print(f"--- [SETUP] Instaluję brakujące pakiety: {', '.join(missing)} ---")
            python = sys.executable
            subprocess.check_call([python, '-m', 'pip', 'install', '-q', '--no-warn-conflicts', *missing], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print("--- [SETUP] Zależności gotowe. ---")
    except Exception as e:
        print(f"--- [SETUP] ❌ Błąd instalacji: {e}. Spróbuj zainstalować pakiety ręcznie.")

ensure_dependencies()

# --- Dalsze importy ---
import pandas as pd
from google.colab import userdata, files
from IPython.display import Markdown, clear_output, display
from tqdm.notebook import tqdm
from Bio import Entrez, Medline

# ===================================================================
# RDZEŃ APLIKACJI - DEFINICJE
# ===================================================================

# --- Moduł Konstytucji Oceny Dowodów ---
class EvidenceAppraisalConstitution:
    ELITE_JOURNALS_MEDICINE = ["N Engl J Med", "Lancet", "JAMA", "BMJ", "Nat Med"]
    ELITE_JOURNALS_PSYCHIATRY = ["Lancet Psychiatry", "World Psychiatry", "JAMA Psychiatry", "Am J Psychiatry", "Mol Psychiatry"]
    SCORING_WEIGHTS = {"GUIDELINE": 100, "SYSTEMATIC_REVIEW": 80, "META_ANALYSIS": 80, "RCT": 60, "NARRATIVE_REVIEW": 20, "OTHER": 10, "ELITE_JOURNAL_BONUS": 50}

# --- Funkcje i Klasy Logiczne ---
def get_ai_keywords(temat, client):
    prompt = f'Analyze topic: "{temat}". Generate JSON with "core_concept_keywords" (specific essence of topic) and "pico_keywords" (hybrid MeSH and tiab terms for P,I,C,O).'
    try:
        r = client.chat.completions.create(model="gpt-4o", messages=[{"role": "system", "content": "You are a PubMed search strategy expert. Respond with JSON."}, {"role": "user", "content": prompt}], response_format={"type": "json_object"})
        return json.loads(r.choices[0].message.content)
    except Exception as e: print(f"❌ BŁĄD OpenAI (Keywords): {e}"); return None

def build_query(ai_result):
    core = " OR ".join(ai_result.get("core_concept_keywords", []))
    pico_groups = [" OR ".join(terms) for _, terms in ai_result.get("pico_keywords", {}).items() if terms]
    full_query = " AND ".join(filter(None, [f"({core})"] + [f"({group})" for group in pico_groups]))
    return f"({full_query}) AND (english[Language])" if full_query else ""

def get_pubmed_count(query):
    try:
        handle = Entrez.esearch(db="pubmed", term=query, retmode="xml"); return int(Entrez.read(handle)["Count"])
    except Exception: return -1

def refine_query(v1, diagnosis, client):
    prompt = f'Initial query (v1): ```{v1}```\nDiagnosis: "{diagnosis}"\nRewrite the query to v2 to fix these issues. Be more specific. Respond with JSON: {{"refined_query": "..."}}'
    try:
        r = client.chat.completions.create(model="gpt-4o", messages=[{"role": "system", "content": "You are a PubMed query optimization expert. Respond with JSON."}, {"role": "user", "content": prompt}], response_format={"type": "json_object"})
        return json.loads(r.choices[0].message.content).get("refined_query")
    except Exception: return v1

def search_and_parse_pubmed(query, topic):
    try:
        Entrez.email = "rafal.jaeschke@gmail.com"; print("\n--- Komunikacja z PubMed ---")
        h_search = Entrez.esearch(db="pubmed", term=query); ids = Entrez.read(h_search)["IdList"]
        if not ids: print("✔ Znaleziono 0 publikacji."); return None
        print(f"✔ Znaleziono {len(ids)} publikacji. Pobieram rekordy...")
        h_fetch = Entrez.efetch(db="pubmed", id=ids, rettype="medline", retmode="text"); raw = h_fetch.read(); h_fetch.close()
        all_data = [{'PMID': r.get('PMID','N/A'), 'Title': r.get('TI','N/A'), 'Abstract': r.get('AB','N/A'), 'Authors': r.get('AU',[]), 'Journal': r.get('TA','N/A'), 'Year': int(r.get('DP','1900')[:4]), 'PubType': r.get('PT',[]), 'Volume': r.get('VI',''), 'Issue': r.get('IP',''), 'Pages': r.get('PG','')} for r in Medline.parse(io.StringIO(raw))]
        return pd.DataFrame(all_data)
    except Exception as e: print(f"❌ BŁĄD PubMed: {e}"); return None

def score_and_rank_articles(df: pd.DataFrame):
    print("\n--- Rozpoczynam Hierarchiczną Ocenę Dowodów ---")
    scores = []; constitution = EvidenceAppraisalConstitution()
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Ocena EBM"):
        score = 0; pub_type = [pt.lower() for pt in row['PubType']]
        if 'guideline' in pub_type: score += constitution.SCORING_WEIGHTS["GUIDELINE"]
        elif 'systematic review' in pub_type or 'meta-analysis' in pub_type: score += constitution.SCORING_WEIGHTS["SYSTEMATIC_REVIEW"]
        elif 'randomized controlled trial' in pub_type: score += constitution.SCORING_WEIGHTS["RCT"]
        elif 'review' in pub_type: score += constitution.SCORING_WEIGHTS["NARRATIVE_REVIEW"]
        else: score += constitution.SCORING_WEIGHTS["OTHER"]
        if row['Journal'] in constitution.ELITE_JOURNALS_MEDICINE or row['Journal'] in constitution.ELITE_JOURNALS_PSYCHIATRY: score += constitution.SCORING_WEIGHTS["ELITE_JOURNAL_BONUS"]
        recency_factor = max(0.5, 1 - (datetime.now().year - row['Year']) / 20)
        score *= recency_factor
        scores.append(score)
    df['ebm_score'] = scores
    df_sorted = df.sort_values(by='ebm_score', ascending=False)
    print("✔ Oceniono i wybrano Top 15 artykułów.")
    return df_sorted.head(15)

def generate_final_report(df_top, question, client):
    display(Markdown("--- \n### ✍️ Konfiguracja Raportu Finalnego"))
    report_type = "For Patients" if input("➡️ Typ raportu: (1) Professional, (2) For Patients: ").strip() == "2" else "Professional"
    language = "Polish" if input("➡️ Język: (1) Polski, (2) Angielski: ").strip() == "1" else "English"
    style_choice = input("➡️ Użyć domyślnego stylu bibliografii (Vancouver)? (tak/nie): ").lower().strip()
    if style_choice in ["tak", "t", ""]: citation_style = "Use the Vancouver citation style."
    else: citation_style = input("➡️ Wklej przykład cytowania lub wytyczne dla autorów: ")

    abstracts_context = "\n\n---\n\n".join(df_top.apply(lambda r: f"Title: {r['Title']}\nAbstract: {r['Abstract']}", axis=1).tolist())
    df_biblio = df_top[['Authors', 'Title', 'Journal', 'Year', 'Volume', 'Issue', 'Pages']].copy()
    df_biblio['Authors'] = df_biblio['Authors'].apply(lambda x: ", ".join(x))
    biblio_json = df_biblio.to_json(orient='records', indent=2)

    instruction_map = {"Professional": "Write a concise, professional literature review summary (Introduction, Key Findings, Limitations, Conclusion).", "For Patients": "Write an accessible summary for a non-medical audience ('What was the research about?', 'What did scientists discover?', 'What does this mean for patients?', 'Limitations')."}
    final_prompt = f"Task: {instruction_map[report_type]}\nReport Language: {language}\nResearch Question: \"{question}\"\nAbstracts for Analysis: {abstracts_context}\n---\nBIBLIOGRAPHY INSTRUCTIONS:\nFormat citations for the articles below according to the specified style.\nARTICLE DATA (JSON):\n{biblio_json}\nCITATION STYLE:\n{citation_style}"

    try:
        print("\n🧠 AI generuje finalny raport i bibliografię. To może potrwać chwilę...")
        r = client.chat.completions.create(model="gpt-4o", messages=[{"role": "system", "content": f"You are an expert medical writer. Your response must be in {language}."}, {"role": "user", "content": final_prompt}], temperature=0.5)
        report_content = r.choices[0].message.content
        display(Markdown(f"---\n## 📄 Wygenerowany Raport\n\n{report_content}"))

        ts = datetime.now().strftime("%Y%m%d"); report_filename = f"raport_{ts}_{report_type.lower()}_{language.lower()}.md"
        with open(report_filename, "w", encoding='utf-8') as f: f.write(report_content)
        print(f"\n⬇️ Inicjuję pobieranie raportu ({report_filename})...")
        files.download(report_filename)
    except Exception as e: print(f"❌ BŁĄD AI (Report Gen): {e}")

# --- Główny Procesor ---
class AutonomousAgent:
    def __init__(self):
        try:
            self.client = openai.OpenAI(api_key=userdata.get("OPENAI_API_KEY"));
            print("✔ Środowisko gotowe (Agent Autonomiczny).")
        except Exception as e: self.client = None; print(f"❌ BŁĄD KRYTYCZNY Inicjalizacji: {e}")

    def run(self, topic: str):
        if not self.client: return
        display(Markdown(f"# 🤖 Autonomiczny Agent Badawczy\n**Temat:** {topic}"))
        ai_res = get_ai_keywords(topic, self.client)
        if not ai_res: return
        question = ai_res.get('structured_question', topic)
        display(Markdown(f"**Sformułowane pytanie:** *{question}*"))
        query = build_query(ai_res)
        display(Markdown(f"**Wygenerowana kwerenda:**\n```\n{query}\n```"))

        df_all = search_and_parse_pubmed(query, topic)
        if df_all is None or df_all.empty: print("Zakończono. Nie znaleziono publikacji do analizy."); return

        df_top15 = score_and_rank_articles(df_all)
        generate_final_report(df_top15, question, self.client)

# --- URUCHOMIENIE ---
if "google.colab" in sys.modules:
    # Używamy zmiennej TEMAT_BADAWCZY z formularza na górze komórki
    if 'TEMAT_BADAWCZY' in locals() and TEMAT_BADAWCZY:
        agent = AutonomousAgent()
        if agent.client:
            agent.run(topic=TEMAT_BADAWCZY)
    else:
        print("❌ BŁĄD: Zmienna TEMAT_BADAWCZY nie jest ustawiona. Wpisz temat w formularzu na górze komórki.")

--- [SETUP] Weryfikacja zależności... ---
--- [SETUP] Zależności gotowe. ---
✔ Środowisko gotowe (Agent Autonomiczny).


# 🤖 Autonomiczny Agent Badawczy
**Temat:** medicinal marihuana for ADHD in adults

**Sformułowane pytanie:** *medicinal marihuana for ADHD in adults*

**Wygenerowana kwerenda:**
```
((medicinal marijuana OR ADHD OR adults) AND (ADHD OR Attention Deficit Hyperactivity Disorder OR adults OR adult population) AND (medicinal marijuana OR medical cannabis OR cannabis-based medicine OR cannabinoids OR THC OR CBD) AND (placebo OR standard treatment OR traditional ADHD medication OR behavioral therapy OR psychostimulants) AND (symptom improvement OR attention enhancement OR reduced hyperactivity OR adverse effects OR treatment efficacy)) AND (english[Language])
```


--- Komunikacja z PubMed ---
✔ Znaleziono 20 publikacji. Pobieram rekordy...

--- Rozpoczynam Hierarchiczną Ocenę Dowodów ---


...

➡️ Typ raportu: (1) Professional, (2) For Patients: 1
➡️ Język: (1) Polski, (2) Angielski: 2
➡️ Użyć domyślnego stylu bibliografii (Vancouver)? (tak/nie): t

🧠 AI generuje finalny raport i bibliografię. To może potrwać chwilę...


---
## 📄 Wygenerowany Raport

Introduction:
The use of medicinal marijuana for managing symptoms of Attention-Deficit/Hyperactivity Disorder (ADHD) in adults is gaining interest due to its potential therapeutic benefits. As ADHD continues to affect a significant portion of the adult population, exploring alternative treatments such as cannabinoids, particularly cannabidiol (CBD) and tetrahydrocannabinol (THC), is crucial. This literature review aims to summarize the current research on the efficacy, safety, and limitations of medicinal marijuana for ADHD in adults.

Key Findings:
1. Cannabinoid Effects: Some studies suggest that cannabinoids, particularly CBD, may have therapeutic effects in managing ADHD symptoms due to their interaction with the endocannabinoid system, which plays a role in regulating mood and attention (Campanale et al., 2025).
2. Pain and Anxiety: While cannabinoids have shown efficacy in managing pain and anxiety, their specific effects on ADHD symptoms require further investigation (Bergeria et al., 2025; Messina et al., 2025).
3. Safety Concerns: Research indicates potential adverse effects, such as liver enzyme elevation and psychomimetic toxicity, especially at higher doses of CBD and THC (Florian et al., 2025; Hardy et al., 2025).
4. Neurobiological Insights: Cannabinoids may modulate brain activity related to craving and addiction, which could be relevant for ADHD management (Murphy et al., 2025).

Limitations:
1. Lack of Direct Research: The abstracts analyzed do not directly address the use of medicinal marijuana specifically for ADHD, highlighting a gap in the literature.
2. Small Sample Sizes: Many studies involve small participant numbers, limiting the generalizability of findings (Hurzeler et al., 2025).
3. Short Study Durations: Most studies are short-term, providing limited information on long-term efficacy and safety (Vetter et al., 2025).

Conclusion:
Current literature provides preliminary insights into the potential benefits and risks of medicinal marijuana for ADHD in adults, but direct evidence remains scarce. Future research should focus on large-scale, long-term studies specifically targeting ADHD to better understand the therapeutic potential and safety of cannabinoids in this population. Until then, clinicians should exercise caution and consider existing evidence when discussing medicinal marijuana as a treatment option for ADHD.

---

Citations:
1. Bergeria CL, Mun CJ, Speed TJ, Huhn AS, Wolinsky D, Vandrey R, Campbell CM, Dunn KE. A within-subject, double-blind, placebo-controlled randomized evaluation of the combined effects of cannabidiol and hydromorphone in a human laboratory pain model. Pain. 2025;166(9):e175-e184.
2. Hurzeler T, DeMayo M, Logge W, Watt J, McGregor IS, Suraev A, Haber P, Morley K. The effect of cannabidiol on neurometabolite levels in alcohol use disorder. Alcohol Alcohol. 2025;60(4).
3. Hardy JR, Greer RM, Pelecanos AM, Huggett GE, Kearney AM, Gurgenci TH, Good PD. Medicinal cannabis for symptom control in advanced cancer: a double-blind, placebo-controlled, randomised clinical trial of 1:1 tetrahydrocannabinol and cannabidiol. Support Care Cancer. 2025;33(8):715.
4. Florian J, Salcedo P, Burkhart K, Shah A, Chekka LMS, Keshishi D, Patel V, Yang S, Fein M, DePalma R, Matta M, Strauss DG, Rouse R. Cannabidiol and Liver Enzyme Level Elevations in Healthy Adults: A Randomized Clinical Trial. JAMA Intern Med. 2025;185(9):1070-1078.
5. Murphy E, Dakhili AH, Ganesan S, Zalesky A, Glarin R, Thomson H, Paloubis A, Kamboj SK, Moffat BA, Poudel G, Suo C, Lorenzetti V. CannChange: a protocol for a feasibility study using fMRI-based neurofeedback to change the neurobiology of craving in cannabis use disorder. BMJ Open. 2025;15(8):e105854.
6. Messina A, Monda A, Vassallo V, Di Maio G, Polito R, La Marra M, Allocca S, Casillo M, Moscatelli F, Scavone C, Taturi F, Monda V, Messina G, Schiraldi C, Monda M. The Potential Benefits of a Novel Food Supplement Based on Cannabis Sativa, Boswellia, and Fish Oil for Pain and Inflammation in Physical Activity: Unraveling the Role of Orexin-A Modulation. Sports (Basel). 2025;13(7).


⬇️ Inicjuję pobieranie raportu (raport_20250905_professional_english.md)...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>